In [1]:
%cd ..

/Users/kevin/Desktop/Abroad/GaTech/master/Study/3rd_Semester/Warehousing/proj/Code/Layout-Optimizer-ISYE-6202-Project
/Users/kevin/Desktop/Abroad/GaTech/master/Study/3rd_Semester/Warehousing/proj/Code/Layout-Optimizer-ISYE-6202-Project


In [2]:
import pandas as pd
import numpy as np
from DataLoader import *
from gurobipy import *
from Model.generic_model import *

In [3]:
data_path = "../WHAI-provided_data/"
p_path = data_path + "02_picking-activity_K1.csv"
i_path = data_path + "04_Item-Master_K1.xlsx"
clayout_path = data_path + "Current_layout.xlsx"
stack_path = data_path + "stack.csv"

picking = read_picking(p_path, i_path)
Clayout = read_clayout(clayout_path)
model_input = generate_input(picking, Clayout)

In [8]:
D = dist_matrix()
D

,x,y,dist,clayout
0,0,0,15.2,-1
1,0,1,14.0,-1
2,0,2,12.8,-1
3,0,3,14.0,-1
4,0,4,15.2,-1
...,...,...,...,...
427,11,31,56.4,-1
428,11,32,57.6,94
429,11,33,58.8,248
430,11,34,60.0,-1


,x,y,dist,clayout
0,0,0,15.2,-1
1,0,1,14.0,-1
2,0,2,12.8,-1
3,0,3,14.0,-1
4,0,4,15.2,-1
...,...,...,...,...
427,11,31,56.4,-1
428,11,32,57.6,94
429,11,33,58.8,248
430,11,34,60.0,-1


# Set Define

In [9]:
D[D["clayout"]!=-1]

,x,y,dist,clayout
11,0,11,23.6,0
23,0,23,38.0,287
35,0,35,52.4,289
47,1,11,24.4,321
59,1,23,38.8,249
...,...,...,...,...
425,11,29,54.0,344
426,11,30,55.2,205
428,11,32,57.6,94
429,11,33,58.8,248


,x,y,dist,clayout
11,0,11,23.6,0
23,0,23,38.0,287
35,0,35,52.4,289
47,1,11,24.4,321
59,1,23,38.8,249
...,...,...,...,...
425,11,29,54.0,344
426,11,30,55.2,205
428,11,32,57.6,94
429,11,33,58.8,248


In [10]:
C = D[D["clayout"]!=-1][["x","y"]].to_records(index=False)
C = np.array(C).astype(tuple)


In [11]:
D_c = dict(zip(C,np.array(D["dist"])))

In [12]:
S = []
with open("output/selection.txt", "r") as f:
    for line in f:
        S.append(int(line))

S = np.array(S)

In [13]:
input_S = model_input[model_input["Artikelno"].isin(S)]
input_S

,Artikelno,Unit_load,Total,Picks,Full_pallet,Min_area,Assigned
0,1,57,1448,621,26,1,0
2,3,420,32564,1620,78,1,0
5,6,182,5159,550,29,1,0
7,8,48,843,351,18,1,0
8,9,133,15476,2427,117,1,0
...,...,...,...,...,...,...,...
378,388,161,6274,794,39,1,0
381,391,75,1486,411,20,1,0
384,394,75,1059,335,15,1,0
389,399,56,2324,241,42,1,0


,Artikelno,Unit_load,Total,Picks,Full_pallet,Min_area,Assigned
0,1,57,1448,621,26,1,0
2,3,420,32564,1620,78,1,0
5,6,182,5159,550,29,1,0
7,8,48,843,351,18,1,0
8,9,133,15476,2427,117,1,0
...,...,...,...,...,...,...,...
378,388,161,6274,794,39,1,0
381,391,75,1486,411,20,1,0
384,394,75,1059,335,15,1,0
389,399,56,2324,241,42,1,0


In [14]:
D_s=dict(zip(input_S["Artikelno"],input_S["Total"]))


In [15]:
P_s=dict(zip(input_S["Artikelno"],input_S["Picks"]))


In [21]:
stack_data = pd.read_csv(stack_path)
stack = read_stack(stack_path,S)
T_s = dict(zip(stack["Artikelno"],stack["Easiness"]))

